In [7]:
import numpy as np
import pandas as pd
import warnings
import itertools

In [2]:
trip_data_csv = "../data/challenge-q/trip_data_3.csv"
trip_fare_csv = "../data/challenge-q/trip_fare_3.csv"
chunksize = 10**5

In [3]:
trip_data_reader = pd.read_csv(trip_data_csv, chunksize=chunksize)
trip_fare_reader = pd.read_csv(trip_fare_csv, chunksize=chunksize)

In [77]:
trip_df = trip_data_reader.get_chunk(10)
trip_df.head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,EC357E5043D2B57A59695A9EBEF0DF66,57E230DE0A42F3A879BE0E9B59E5774D,CMT,1,N,2013-03-01 00:04:57,2013-03-01 00:15:39,2,641,2.0,-73.988785,40.722244,-73.979820,40.746365
1,9202653FEF132ECCF2850C7E560CDE08,A86011E8B4D0FCC19283230950DFCB3E,CMT,1,N,2013-03-01 00:05:04,2013-03-01 00:11:25,1,381,1.6,-73.960480,40.776127,-73.981209,40.780640
2,8B8D3A06CE1FAD14B5ADBE5BE40C77CC,9BC5440E1CDC10E1AF07207A1A315C23,CMT,1,N,2013-03-01 00:05:14,2013-03-01 00:19:28,1,853,6.6,-74.008919,40.713947,-73.916862,40.742718
3,46E1AE760E9E1FC1573B878C4FB9062A,C21B0744AFE2EB82768C26782238B35F,CMT,1,N,2013-03-01 00:05:14,2013-03-01 00:47:14,1,2519,22.0,-74.000793,40.729679,-73.786713,40.843445
4,3CD865423995A255E87078AFBE87429C,69365235F0D552B411CDD6D7980C08F5,CMT,1,N,2013-03-01 00:05:23,2013-03-01 00:20:41,1,917,3.8,-73.960159,40.762028,-74.004257,40.738377


In [76]:
trip_fare_reader = pd.read_csv(trip_fare_csv, chunksize=chunksize)
fare_df = trip_fare_reader.get_chunk(10)
fare_df.head()

,medallion,hack_license,vendor_id,pickup_datetime,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,FE7B354FEB67B9C94BD34EA54469691C,2C78614ADC9C602EC70D65CCB4E63B14,CMT,2013-03-01 00:00:04,CRD,39.5,0.5,0.5,10.10,0.0,50.60
1,B1370E6501ED61CEDA0160199AF7708C,08BF18740B38D64720C9E060429A3968,CMT,2013-03-01 00:00:18,CRD,13.0,0.5,0.5,2.10,0.0,16.10
2,CBC20164C22B96CAF5BC86C49F749DBB,F3310C8DA3DB753DDB5800F32D3CB8E3,CMT,2013-03-01 00:00:18,CRD,15.0,0.5,0.5,3.20,0.0,19.20
3,B8256802A449BEF34E161654BFA8A2B6,DF142C5256392C3CDA67C7DFFA5B88E2,CMT,2013-03-01 00:00:23,CRD,25.5,0.5,0.5,6.26,4.8,37.56
4,0F9E0728AB1E40D5CEB0C6EDBF805CCB,10D2E58D75E07D6B5AAFA329BF5A4CAC,CMT,2013-03-01 00:00:35,CRD,6.5,0.5,0.5,1.00,0.0,8.50


In [59]:
def df_query_count(fare_df, querystr1, querystr2):
    # regularize the column header names, drop leading and trailing spaces
    fare_df.columns = [cn.strip() for cn in fare_df.columns]
    fare_df = fare_df.dropna()
    n_rows_1 = len(fare_df.query(querystr1))
    n_rows_1_2 = len(fare_df.query(querystr1+' and '+querystr2))
    return n_rows_1, n_rows_1_2

In [61]:
querystr1 = 'fare_amount<5'
querystr2 = 'payment_type=="CRD"'
count = np.array([0, 0])
count_all = [ df_query_count(chunk, querystr1, querystr2) for chunk in trip_fare_reader ]
count_1, count_1_2 = zip(*count_all)

AttributeError: 'numpy.int64' object has no attribute '3f'

In [65]:
ratio = np.sum(np.array(count_1_2,dtype=float))/np.sum(np.array(count_1, dtype=float))
print "Q2.1, What fraction of payments under $5 use a credit card?\nAnswer = {0:3f}".format(ratio)

Q2.1, What fraction of payments under $5 use a credit card?
Answer = 0.367564


In [72]:
trip_fare_reader = pd.read_csv(trip_fare_csv, chunksize=chunksize)
querystr1 = 'fare_amount>50'
querystr2 = 'payment_type=="CRD"'
count = np.array([0, 0])
count_all = [ df_query_count(chunk, querystr1, querystr2) for chunk in trip_fare_reader ]
count_1, count_1_2 = zip(*count_all)
ratio = np.sum(np.array(count_1_2,dtype=float))/np.sum(np.array(count_1, dtype=float))
print "Q2.2, What fraction of payments over $50 use a credit card\nAnswer = {0:3f}".format(ratio)

Q2.2, What fraction of payments over $50 use a credit card
Answer = 0.622889


In [9]:
trip_data_reader = pd.read_csv(trip_data_csv, chunksize=chunksize)
trip_fare_reader = pd.read_csv(trip_fare_csv, chunksize=chunksize)
trip_df = trip_data_reader.get_chunk(10)
fare_df = trip_fare_reader.get_chunk(10)
trip_df.columns = [ cn.strip() for cn in trip_df.columns]
fare_df.columns = [ cn.strip() for cn in fare_df.columns]

In [14]:
trip_df.columns

Index([u'medallion', u'hack_license', u'vendor_id', u'rate_code', u'store_and_fwd_flag', u'pickup_datetime', u'dropoff_datetime', u'passenger_count', u'trip_time_in_secs', u'trip_distance', u'pickup_longitude', u'pickup_latitude', u'dropoff_longitude', u'dropoff_latitude'], dtype='object')

In [13]:
fare_df.isnull(

Index([u'medallion', u'hack_license', u'vendor_id', u'pickup_datetime', u'payment_type', u'fare_amount', u'surcharge', u'mta_tax', u'tip_amount', u'tolls_amount', u'total_amount'], dtype='object')

In [24]:
def fare_per_unit(trip_df, fare_df):
    trip_df.columns = [ cn.strip() for cn in trip_df.columns]
    fare_df.columns = [ cn.strip() for cn in fare_df.columns]
    # data check warning
    if not np.all( np.logical_and.reduce((trip_df['medallion']==fare_df['medallion'], \
                                          trip_df['hack_license']==fare_df['hack_license'], \
                                          trip_df['pickup_datetime']==fare_df['pickup_datetime'])) ):
        warnings.warn("Trip data and trip fare are not one-to-one correspondent! Be cautious about your results")
    select_flag = ~np.logical_or(trip_df.isnull().any(axis=1), fare_df.isnull().any(axis=1))
    trip_df = trip_df.loc[select_flag, :]
    fare_df = fare_df.loc[select_flag, :]
    select_flag = np.logical_and.reduce((trip_df['medallion']==fare_df['medallion'], \
                                         trip_df['hack_license']==fare_df['hack_license'], \
                                         trip_df['pickup_datetime']==fare_df['pickup_datetime'], \
                                         trip_df['trip_time_in_secs']>1e-10, \
                                         trip_df['trip_distance']>1e-10))
    trip_df = trip_df.loc[select_flag, :]
    fare_df = fare_df.loc[select_flag, :]
    n_trips = len(trip_df)
    fare_minute_sum = np.sum(fare_df['fare_amount']/(trip_df['trip_time_in_secs']/60.0))
    fare_mile_med = np.median(fare_df['fare_amount']/(trip_df['trip_distance']))
    return n_trips, fare_minute_sum, fare_mile_med

In [101]:
fare_minute_sum

12.63872671443756

In [25]:
trip_data_reader = pd.read_csv(trip_data_csv, chunksize=chunksize)
trip_fare_reader = pd.read_csv(trip_fare_csv, chunksize=chunksize)
fpu_list = [ fare_per_unit(trip_chunk, fare_chunk) for trip_chunk, fare_chunk in itertools.izip(trip_data_reader, trip_fare_reader)]
n_trips, fare_minute_sum, fare_mile_med = zip(*fpu_list)
n_trips = np.sum(np.array(n_trips))
fare_minute_sum = np.sum(np.array(fare_minute_sum))



/usr3/graduate/zhanli86/anaconda_envs/zhanli_py27/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)


In [29]:
print "Q2.3 What is the mean fare per minute driven?\nAnswer: {0:3f}".format(fare_minute_sum/float(n_trips))
print "Q2.3 What is the median of the taxi's fare per mile driven?\nAnswer: {0:3f}".format(np.median(np.array(fare_mile_med)))

Q2.3 What is the mean fare per minute driven?
Answer: 1.245245
Q2.3 What is the median of the taxi's fare per mile driven?
Answer: 5.000000


In [31]:
from geopy.distance import great_circle

In [32]:
def distance_ratio(trip_df, fare_df):
    trip_df.columns = [ cn.strip() for cn in trip_df.columns]
    fare_df.columns = [ cn.strip() for cn in fare_df.columns]
    # data check warning
    if not np.all( np.logical_and.reduce((trip_df['medallion']==fare_df['medallion'], \
                                          trip_df['hack_license']==fare_df['hack_license'], \
                                          trip_df['pickup_datetime']==fare_df['pickup_datetime'])) ):
        warnings.warn("Trip data and trip fare are not one-to-one correspondent! Be cautious about your results")
    select_flag = ~np.logical_or(trip_df.isnull().any(axis=1), fare_df.isnull().any(axis=1))
    trip_df = trip_df.loc[select_flag, :]
    fare_df = fare_df.loc[select_flag, :]
    select_flag = np.logical_and.reduce((trip_df['medallion']==fare_df['medallion'], \
                                         trip_df['hack_license']==fare_df['hack_license'], \
                                         trip_df['pickup_datetime']==fare_df['pickup_datetime'], \
                                         trip_df['trip_time_in_secs']>1e-10, \
                                         trip_df['trip_distance']>1e-10))
    trip_df = trip_df.loc[select_flag, :]
    fare_df = fare_df.loc[select_flag, :]
    n_trips = len(trip_df)
    if n_trips>0:
        pickup_loc_list = zip(trip_df['pickup_latitude'], trip_df['pickup_longitude'])
        dropoff_loc_list = zip(trip_df['dropoff_latitude'], trip_df['dropoff_longitude'])
        dist_list = [great_circle(pu, do).miles for pu, do in itertools.izip(pickup_loc_list, dropoff_loc_list)]
        dist_ratio = np.array(dist_list)/trip_df['trip_distance']
        dist_ratio_sum = np.sum(dist_ratio)
    else:
        dist_ratio_sum = 0
    return n_trips, dist_ratio_sum

In [33]:
trip_data_reader = pd.read_csv(trip_data_csv, chunksize=chunksize)
trip_fare_reader = pd.read_csv(trip_fare_csv, chunksize=chunksize)
fpu_list = [ distance_ratio(trip_chunk, fare_chunk) for trip_chunk, fare_chunk in itertools.izip(trip_data_reader, trip_fare_reader)]
n_trips, dist_ratio_sum = zip(*fpu_list)
n_trips = np.sum(np.array(n_trips))
dist_ratio_sum = np.sum(np.array(dist_ratio_sum))

In [34]:
print "Q2.6 What is the average ratio of the distance between the pickup and dropoff divided by the distance driven??\nAnswer: {0:3f}".format(dist_ratio_sum/float(n_trips))

Q2.6 What is the average ratio of the distance between the pickup and dropoff divided by the distance driven??
Answer: 4.327638


In [41]:
def tip_from_loc(trip_df, fare_df, pickup_loc):
    trip_df.columns = [ cn.strip() for cn in trip_df.columns]
    fare_df.columns = [ cn.strip() for cn in fare_df.columns]
    # data check warning
    if not np.all( np.logical_and.reduce((trip_df['medallion']==fare_df['medallion'], \
                                          trip_df['hack_license']==fare_df['hack_license'], \
                                          trip_df['pickup_datetime']==fare_df['pickup_datetime'])) ):
        warnings.warn("Trip data and trip fare are not one-to-one correspondent! Be cautious about your results")
    select_flag = ~np.logical_or(trip_df.isnull().any(axis=1), fare_df.isnull().any(axis=1))
    trip_df = trip_df.loc[select_flag, :]
    fare_df = fare_df.loc[select_flag, :]
    select_flag = np.logical_and.reduce((trip_df['medallion']==fare_df['medallion'], \
                                         trip_df['hack_license']==fare_df['hack_license'], \
                                         trip_df['pickup_datetime']==fare_df['pickup_datetime'], \
                                         trip_df['trip_time_in_secs']>1e-10, \
                                         trip_df['trip_distance']>1e-10))
    trip_df = trip_df.loc[select_flag, :]
    fare_df = fare_df.loc[select_flag, :]
    n_trips = len(trip_df)
    if n_trips>0:
        pickup_loc_list = zip(trip_df['pickup_latitude'], trip_df['pickup_longitude'])
        dist_list = [great_circle(pu, pickup_loc).miles for pu in pickup_loc_list]
        dist_list = np.array(dist_list)
        select_flag = dist_list < 5
        n_trips = len(fare_df.loc[select_flag, :])
        tip_sum = np.sum(fare_df.loc[select_flag, 'tip_amount'])
    else:
        tip_sum = 0
    return n_trips, tip_sum

In [42]:
jfk_geoloc = (40.6397, -73.7789)
trip_data_reader = pd.read_csv(trip_data_csv, chunksize=chunksize)
trip_fare_reader = pd.read_csv(trip_fare_csv, chunksize=chunksize)
fpu_list = [ tip_from_loc(trip_chunk, fare_chunk, jfk_geoloc) for trip_chunk, fare_chunk in itertools.izip(trip_data_reader, trip_fare_reader)]
n_trips, tip_sum = zip(*fpu_list)
n_trips = np.sum(np.array(n_trips))
tip_sum = np.sum(np.array(tip_sum))

In [43]:
print "Q2.7 What is the average tip for rides from JFK?\nAnswer: {0:3f}".format(tip_sum/float(n_trips))

Q2.7 What is the average tip for rides from JFK?
Answer: 4.465155


In [44]:
print n_trips
print tip_sum

106064
473592.2
